### Interactive Plotting and Monte Carlo Simulations of CPPI

In [1]:
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
import portfolio_management_lib as prt

%load_ext autoreload
%autoreload 2

##### GBM Simulations

In [2]:
def show_gbm(
    n_scenarios: int,
    mu: float,
    sigma: float,
) -> None:
    """
    Draw the results of a stock price evolution under a Geometric Brownian Motion Model.
    """
    s_0: int = 100
    prices = prt.gbm(
        n_scenarios = n_scenarios,
        mu = mu,
        sigma = sigma,
    )
    ax = prices.plot(
        legend = False,
        color = "indianred",
        alpha = 0.5,
        linewidth = 2,
        figsize = (12, 5),
    )
    ax.axhline(
        y = 100,
        ls = ":",
        color = "black",
    )
    
    # draw a dot at the origin
    ax.plot(0, s_0, marker = "o", color = 'darkred', alpha = 0.2)

    return

In [3]:
gbm_controls = widgets.interactive(
    show_gbm,
    n_scenarios = widgets.IntSlider(
        min = 1, max = 100, step = 1, value = 1,
    ),
    mu = (0., .2, 0.01),
    sigma = (0, .3, 0.01)
)

In [4]:
display(gbm_controls)

interactive(children=(IntSlider(value=1, description='n_scenarios', min=1), FloatSlider(value=0.1, description…

##### Interactive CPPI Monte Carlo Testing

In [5]:
def show_cppi(
    n_scenarios: int = 50,
    mu: float = 0.07,
    sigma: float = 0.15,
    m: float = 3,
    floor: float = 0.,
    riskfree_rate = 0.03,
    y_max = 100,
) -> None:
    """
    Plot the results of a Monte Carlo Simulations of CPPI.
    """
    start = 100
    sim_rets = prt.gbm(
        n_scenarios = n_scenarios,
        mu = mu,
        sigma = sigma,
        s_0 = start,
        steps_per_year = 12,
        prices = False,
    )
    riksy_r = sim_rets

    # run the back-test
    btr = prt.run_cppi(
        risky_r = riksy_r,
        m = m, # multiplier
        start = start,
        floor = floor,
        riskfree_rate = riskfree_rate
    )

    wealth = btr["Wealth"]
    y_max = wealth.values.max() * y_max / 100

    ax = wealth.plot(
        legend = False,
        alpha = 0.3,
        color = "indianred",
        figsize = (12, 6)
    )
    # Start Value
    ax.axhline(
        y = start,
        ls = ":",
        color = "black",
    )
    # Floor Value
    ax.axhline(
        y = start * floor,
        ls = "--",
        color = "red",
    )
    ax.set_ylim(top = y_max)
    return

cppi_controls = widgets.interactive(
    show_cppi,
    n_scenario = widgets.IntSlider(
        min = 1,
        max = 1000,
        step = 5,
        value = 50,
    ),
    mu = (0., .2, 0.01),
    sigma = (0, 0.30, 0.05),
    floor = (0, 2, .1),
    m = (1, 5, .5),
    riskfree_rate = (0, .05, .01),
    y_max = widgets.IntSlider(
        min = 0,
        max = 100,
        step = 1,
        value = 100,
        description = "Zoom Y Axis"
    )
)

display(cppi_controls)

interactive(children=(IntSlider(value=50, description='n_scenarios', max=150, min=-50), FloatSlider(value=0.07…

##### Adding a Histogram and Reporting Floor Violations

In [6]:
import matplotlib.pyplot as plt

def show_cppi(
    n_scenarios: int = 50,
    mu: float = 0.07,
    sigma: float = 0.15,
    m: float = 3.,
    floor: float = 0.0,
    riskfree_rate: float = 0.03,
    y_max: float = 100.0,
) -> None:
    """
    Plot the results of a Monte Carlo Simulation of CPPI
    """
    start = 100
    sim_rets = prt.gbm(
        n_scenarios = n_scenarios,
        mu = mu,
        sigma = sigma,
        prices = False,
    )
    risky_r = sim_rets

    # run the back test
    # backtest result -> btr
    btr = prt.run_cppi(
        risky_r = risky_r,
        m = m,
        start = start,
        floor = floor,
        riskfree_rate = riskfree_rate,
    )
    wealth = btr["Wealth"]
    
    # Caclulate terminal wealth state
    y_max = wealth.values.max() * y_max / 100
    terminal_wealth = wealth.iloc[-1]

    # Plot
    fig, (wealth_ax, hist_ax) = plt.subplots(
        nrows = 1, 
        ncols = 2, 
        sharey = True,
        gridspec_kw = {
            'width_ratios': [3, 2],
        },
        figsize = (24, 9)
    )
    plt.subplots_adjust(wspace = 0.0)

    wealth.plot(
        ax = wealth_ax,
        legend = False,
        alpha = 0.3,
        color = "indianred",
    )

    wealth_ax.axhline(y = start, ls = ":", color = "black")
    wealth_ax.axhline(y = start * floor, ls = "--", color = "red")
    wealth_ax.set_ylim(top = y_max)

    terminal_wealth.plot.hist(ax = hist_ax, bins = 50, ec = 'w', fc = "indianred", orientation = 'horizontal')
    hist_ax.axhline(y = start, ls = ":", color = "black")

    return

cppi_controls = widgets.interactive(
    show_cppi,
    n_scenarios = widgets.IntSlider(min = 1, max = 1000, step = 5, value = 50),
    mu = (0., .2, 0.01),
    floor = (0, 2, .05),
    m = (1, 5, .5),
    riskfree_rate = (0, 0.05, 0.01),
    y_max = widgets.IntSlider(
        min = 0, 
        max = 100,
        step = 1,
        value = 100,
        descrption = "Zoom Y Axis",
    )
)

display(cppi_controls)

interactive(children=(IntSlider(value=50, description='n_scenarios', max=1000, min=1, step=5), FloatSlider(val…

##### Adding Terminal Wealth Statistics

In [8]:
import matplotlib.pyplot as plt
import numpy as np

def show_cppi(
    n_scenarios: int = 50, # number of trajectories
    mu: float = 0.07, # annualized return
    sigma: float = 0.15, # annualized volatility
    m: float = 3,
    floor: float = 0.,
    riskfree_rate: float = 0.03,
    steps_per_year: float = 12,
    y_max: int = 100,
) -> None:
    """
    Plot the results of a Monte Carlo Simulation of CPPI.
    """
    start = 100
    sim_rets = prt.gbm(
        n_scenarios=n_scenarios,
        mu = mu,
        sigma = sigma,
        prices = False,
        steps_per_year = steps_per_year,
    )
    risky_r = pd.DataFrame(sim_rets)

    # run the back test
    btr = prt.run_cppi(
        risky_r = risky_r,
        m = m,
        start = start,
        floor = floor,
        riskfree_rate = riskfree_rate,
    )
    wealth = btr["Wealth"]

    # calculate terminal wealth stats
    y_max = wealth.values.max() * y_max / 100
    terminal_wealth = wealth.iloc[-1]

    tw_mean = terminal_wealth.mean()
    tw_median = terminal_wealth.median()
    failure_mask = np.less(terminal_wealth, start * floor)
    n_failures = failure_mask.sum()
    p_failures = n_failures / n_scenarios

    e_shortfall = np.dot(terminal_wealth - start * floor, failure_mask) / n_failures if n_failures > 0 else 0.

    # plot
    fig, (wealth_ax, hist_ax) = plt.subplots(
        nrows = 1,
        ncols = 2,
        sharey= True,
        gridspec_kw = {
            'width_ratios':[3,2]
        },
        figsize = (24, 9),
    )

    plt.subplots_adjust(wspace = 0.0)

    wealth.plot(ax = wealth_ax, legend = False, alpha = 0.3, color = "indianred")
    # initial asset value
    wealth_ax.axhline(
        y = start,
        ls = ":",
        color = "black" 
    )
    # floor asset value
    wealth_ax.axhline(
        y = start * floor,
        ls = ":",
        color = "red",
    )
    wealth_ax.set_ylim(top = y_max)

    terminal_wealth.plot.hist(
        ax = hist_ax,
        bins = 50,
        ec = 'w',
        fc = 'indianred',
        orientation = 'horizontal'
    )
    hist_ax.axhline(y = start, ls = ":", color = "black")
    hist_ax.axhline(y = tw_mean, ls = ":", color = "blue")
    hist_ax.axhline(y = tw_median, ls = ":", color = "purple")
    hist_ax.annotate(f"Mean: ${int(tw_mean)}", xy=(.7, .9),xycoords='axes fraction', fontsize = 20)
    hist_ax.annotate(f"Median: ${int(tw_median)}", xy = (.7, .85), xycoords='axes fraction', fontsize = 20)

    if (floor > 1e-3):
        hist_ax.axhline(
            y=start*floor,
            ls="--", 
            color="red", 
            linewidth=3
            )
        hist_ax.annotate(
            f"Violations: {n_failures} ({p_failures*100:2.2f}%)\nE(shortfall)=${e_shortfall:2.2f}", 
            xy=(.7, .7), 
            xycoords='axes fraction', 
            fontsize=20
            )

    return

cppi_controls = widgets.interactive(
    show_cppi,
    # n_scenarios=widgets.IntSlider(min=100, max=1000, step=10, value=50),
    n_scenarios = (1, 1000, 10),
    mu=(0., +.2, .01),
    sigma=(0, .3, .05),
    floor=(0, 2, .1),
    m=(1, 5, .5),
    riskfree_rate=(0, .05, .01),
    steps_per_year=widgets.IntSlider(min=1, max=12, step=1, value=12,
    description="Rebals/Year"),
    y_max=widgets.IntSlider(min=0, max=100, step=1, value=100,
    description="Zoom Y Axis")
)
display(cppi_controls)

interactive(children=(IntSlider(value=50, description='n_scenarios', max=1000, min=1, step=10), FloatSlider(va…